# **Predicción de la Deserción Estudiantil en la Universidad Nacional de Ingeniería (2018-2024) con Inteligencia Artificial** 

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns                       #visualisation
import matplotlib.pyplot as plt             #visualisation
%matplotlib inline     
sns.set(color_codes=True)

In [5]:
data_admision = pd.read_csv(r'..\Datos_abiertos_admision_2021_1_2024_1.csv')
data_preuniversitario = pd.read_csv(r'..\Datos_abiertos_cepre.csv')

In [11]:
# data_preuniversitario.dtypes
# data_admision.dtypes
data_admision.dtypes

IDHASH                  object
COLEGIO                 object
COLEGIO_DEPA            object
COLEGIO_PROV            object
COLEGIO_DIST            object
COLEGIO_PAIS            object
COLEGIO_ANIO_EGRESO      int64
ESPECIALIDAD            object
ANIO_POSTULA             int64
CICLO_POSTULA            int64
DOMICILIO_DEPA          object
DOMICILIO_PROV          object
DOMICILIO_DIST          object
ANIO_NACIMIENTO          int64
NACIMIENTO_PAIS         object
NACIMIENTO_DEPA         object
NACIMIENTO_PROV         object
NACIMIENTO_DIST         object
SEXO                    object
CALIF_FINAL            float64
INGRESO                 object
MODALIDAD               object
dtype: object

# **Query in Python Dataframe**

In [13]:
data_admision.groupby("IDHASH").size().head(3)

IDHASH
00014F32BBBF4F714170C971C555A0AE4359AF4099D82C97E56B617EC223F62B5365F58F64D291AA9A6C6F95057BA8B06DCB18F100020F8DB9E6870697552AE3    1
0002DF41C79D5E34153DB578014A6EADFF766488667DC31737EF7CA01A000919C8E20D1D38C01DC079B1A5E8654D453BFDAF47832F0DB24F4DE662D2A38A7721    3
000718555FBD4F283BCAB5C3B8A2CAB20C6522CF82F52C9D31A0AF4E68714ECCFF93D8FFB3D911570C0B219342C0CAEE94DF38C7C9AE0CE8C523DFDFBCCDB97B    1
dtype: int64

In [33]:
merged_inner = pd.merge(left=data_preuniversitario, right=data_admision, how='inner', left_on='IDHASH', right_on='IDHASH')

print("Columnas en merged_inner:", merged_inner.columns)

Columnas en merged_inner: Index(['IDHASH', 'COLEGIO_x', 'COLEGIO_DEPA_x', 'COLEGIO_PROV_x',
       'COLEGIO_DIST_x', 'COLEGIO_PAIS_x', 'COLEGIO_ANIO_EGRESO_x',
       'ESPECIALIDAD_x', 'ANIO_POSTULA_x', 'CICLO_POSTULA_x',
       'DOMICILIO_DEPA_x', 'DOMICILIO_PROV_x', 'DOMICILIO_DIST_x',
       'ANIO_NACIMIENTO_x', 'NACIMIENTO_PAIS_x', 'NACIMIENTO_DEPA_x',
       'NACIMIENTO_PROV_x', 'NACIMIENTO_DIST_x', 'SEXO_x', 'CALIF_FINAL_x',
       'INGRESO_x', 'MODO_INGRESO', 'FECHA_CORTE', 'COLEGIO_y',
       'COLEGIO_DEPA_y', 'COLEGIO_PROV_y', 'COLEGIO_DIST_y', 'COLEGIO_PAIS_y',
       'COLEGIO_ANIO_EGRESO_y', 'ESPECIALIDAD_y', 'ANIO_POSTULA_y',
       'CICLO_POSTULA_y', 'DOMICILIO_DEPA_y', 'DOMICILIO_PROV_y',
       'DOMICILIO_DIST_y', 'ANIO_NACIMIENTO_y', 'NACIMIENTO_PAIS_y',
       'NACIMIENTO_DEPA_y', 'NACIMIENTO_PROV_y', 'NACIMIENTO_DIST_y', 'SEXO_y',
       'CALIF_FINAL_y', 'INGRESO_y', 'MODALIDAD'],
      dtype='object')
